# Mount Drive & Setup môi trường

In [ ]:
from google.colab import drive
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Định nghĩa đường dẫn (Sửa lại cho đúng folder của bạn)
PROJECT_ROOT = '/content/drive/MyDrive/RNNoise_Project'
SRC_DIR = os.path.join(PROJECT_ROOT, 'src')
TORCH_DIR = os.path.join(PROJECT_ROOT, 'torch', 'rnnoise')
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')

# 3. Cài đặt thư viện build (Thường Colab có sẵn, chạy cho chắc)
!apt-get update && apt-get install -y autoconf automake libtool build-essential

# Compile code C (để có tool dump_features)

In [ ]:
%cd {SRC_DIR}

# Cấp quyền thực thi
!chmod +x autogen.sh configure

# Build "nhà máy" chế biến dữ liệu
!./autogen.sh
!./configure
!make clean # Clean trước cho sạch
!make

# Kiểm tra xem đã đẻ ra file dump_features chưa
if os.path.exists("dump_features"):
    print("✅ Đã build xong dump_features!")
else:
    print("❌ Lỗi build! Kiểm tra log bên trên.")

# Sinh dữ liệu Training (.f32)

In [ ]:
%cd {SRC_DIR}

# Định nghĩa input/output
speech_pcm = os.path.join(DATA_DIR, 'speech.pcm')
noise_pcm = os.path.join(DATA_DIR, 'noise.pcm')
fg_noise_pcm = os.path.join(DATA_DIR, 'fg_noise.pcm')
output_f32 = os.path.join(DATA_DIR, 'training_data.f32')

# Số lượng mẫu (Ví dụ 100,000 cho nhanh, muốn khôn thì tăng lên 500,000)
COUNT = 100000

# Chạy lệnh dump (Lưu ý: PyTorch version vẫn dùng tool C này)
!./dump_features "{speech_pcm}" "{noise_pcm}" "{fg_noise_pcm}" "{output_f32}" {COUNT}

print(f"🎉 Đã sinh xong file: {output_f32}")

## Train với PyTorch

In [ ]:
%cd {TORCH_DIR}

# Cài dependency nếu file requirements.txt có yêu cầu lạ (thường thì torch có sẵn rồi)
# !pip install -r ../weight-exchange/requirements.txt

# Input: File .f32 vừa sinh
# Output: Folder chứa weight
DATA_F32 = os.path.join(DATA_DIR, 'training_data.f32')
OUTPUT_DIR = os.path.join(DATA_DIR, 'model_checkpoints')

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

print("🚀 Bắt đầu train với PyTorch...")
# Cấu trúc lệnh thường thấy của bản PyTorch này:
# python train_rnnoise.py <path_to_f32> <output_dir>
!python train_rnnoise.py "{DATA_F32}" "{OUTPUT_DIR}" --epochs 50 --batch-size 64

# Lưu ý: Kiểm tra lại help nếu chạy lỗi tham số
# !python train_rnnoise.py --help

## Xuất ra header C (.h) cho Nhúng

In [ ]:
%cd {TORCH_DIR}

# Tìm file model mới nhất vừa train xong
# (Giả sử nó lưu tên là weights_ep50.pth, bạn thay tên file thực tế vào)
MODEL_PTH = os.path.join(OUTPUT_DIR, "weights_final.pth")

# Chạy script export (Check file dump_rnnoise_weights.py trong folder torch xem tham số chính xác)
!python dump_rnnoise_weights.py "{MODEL_PTH}" rnnoise_data.c

print("Xong! File rnnoise_data.c đã sẵn sàng để nhúng vào mạch.")